In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

In [25]:
chem = pd.read_csv("F:/PML/Cases/Chemical_Process_Data/ChemicalProcess.csv")
chem

,Yield,BiologicalMaterial01,BiologicalMaterial02,BiologicalMaterial03,BiologicalMaterial04,BiologicalMaterial05,BiologicalMaterial06,BiologicalMaterial07,BiologicalMaterial08,BiologicalMaterial09,...,ManufacturingProcess36,ManufacturingProcess37,ManufacturingProcess38,ManufacturingProcess39,ManufacturingProcess40,ManufacturingProcess41,ManufacturingProcess42,ManufacturingProcess43,ManufacturingProcess44,ManufacturingProcess45
0,38.00,6.25,49.58,56.97,12.74,19.51,43.73,100.0,16.66,11.44,...,0.019,0.5,3,7.2,NaN,NaN,11.6,3.0,1.8,2.4
1,42.44,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,...,0.019,2.0,2,7.2,0.1,0.15,11.1,0.9,1.9,2.2
2,42.03,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,...,0.018,0.7,2,7.2,0.0,0.00,12.0,1.0,1.8,2.3
3,41.42,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,...,0.018,1.2,2,7.2,0.0,0.00,10.6,1.1,1.8,2.1
4,42.49,7.47,63.33,72.25,14.02,17.91,54.66,100.0,18.22,12.80,...,0.017,0.2,2,7.3,0.0,0.00,11.0,1.1,1.7,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,39.66,6.71,56.32,66.19,12.35,20.02,50.26,100.0,17.54,12.50,...,NaN,2.3,0,0.0,0.0,0.00,0.0,0.6,0.0,0.0
172,39.68,6.87,56.74,66.61,12.55,20.18,50.80,100.0,17.48,12.41,...,NaN,1.0,0,0.0,0.0,0.00,0.0,0.6,0.0,0.0
173,42.23,7.50,58.41,68.30,13.33,20.81,52.96,100.0,17.23,12.04,...,NaN,1.3,0,0.0,0.0,0.00,0.0,0.6,0.0,0.0
174,38.48,7.53,58.36,69.25,14.35,20.57,51.31,100.0,17.87,12.77,...,NaN,2.3,0,0.0,0.0,0.00,0.0,0.5,0.0,0.0


In [26]:
X = chem.drop('Yield', axis=1)
y = chem['Yield']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=23)

In [30]:
imp = SimpleImputer(strategy='mean').set_output(transform="pandas")
X_trn_imp = imp.fit_transform(X_train)
X_tst_imp = imp.transform(X_test)

In [34]:
elastic = ElasticNet(alpha=0.001, l1_ratio=0.5)
elastic.fit(X_trn_imp, y_train)
y_pred = elastic.predict(X_tst_imp)
print(r2_score(y_test,y_pred))

0.45009080377831334


C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.371e+01, tolerance: 4.532e-02
  model = cd_fast.enet_coordinate_descent(


#### Using pipeline


In [35]:
from sklearn.pipeline import Pipeline

In [37]:
pipe = Pipeline([('IMP',imp), ('ELASTIC', elastic)])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))

0.45009080377831334


C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.371e+01, tolerance: 4.532e-02
  model = cd_fast.enet_coordinate_descent(


#### kfold cv

In [41]:
kfold = KFold(n_splits=5, shuffle=True, random_state=23)
results = cross_val_score(pipe, X, y, cv = kfold)
print(results.mean())

-4.129321975653751


C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.939e+01, tolerance: 4.974e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.475e+01, tolerance: 5.010e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

#### grid search cv

In [45]:
pipe = Pipeline([('IMP',imp), ('ELASTIC', elastic)])
print(pipe.get_params())
params = {'ELASTIC__alpha':[0.001,0.1,1,1.5,2,5],
         'ELASTIC__l1_ratio':[0,0.25,0.5,0.75,1]}
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold)
gcv.fit(X,y)
print(gcv.best_score_)
print(gcv.best_params_)

{'memory': None, 'steps': [('IMP', SimpleImputer()), ('ELASTIC', ElasticNet(alpha=0.001))], 'verbose': False, 'IMP': SimpleImputer(), 'ELASTIC': ElasticNet(alpha=0.001), 'IMP__add_indicator': False, 'IMP__copy': True, 'IMP__fill_value': None, 'IMP__keep_empty_features': False, 'IMP__missing_values': nan, 'IMP__strategy': 'mean', 'ELASTIC__alpha': 0.001, 'ELASTIC__copy_X': True, 'ELASTIC__fit_intercept': True, 'ELASTIC__l1_ratio': 0.5, 'ELASTIC__max_iter': 1000, 'ELASTIC__positive': False, 'ELASTIC__precompute': False, 'ELASTIC__random_state': None, 'ELASTIC__selection': 'cyclic', 'ELASTIC__tol': 0.0001, 'ELASTIC__warm_start': False}


C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.913e+01, tolerance: 4.974e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.423e+01, tolerance: 5.010e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers im

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.225e+01, tolerance: 4.974e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.538e+01, tolerance: 5.010e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.248e+01, tolerance: 4.974e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.395e+01, tolerance: 5.010e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers im

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.881e+01, tolerance: 5.010e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.809e+01, tolerance: 4.554e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

0.43336060693136014
{'ELASTIC__alpha': 2, 'ELASTIC__l1_ratio': 0}


C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.584e+01, tolerance: 5.010e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.122e+01, tolerance: 4.554e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

In [48]:
print(gcv.best_score_)
print(gcv.best_params_)

0.43336060693136014
{'ELASTIC__alpha': 2, 'ELASTIC__l1_ratio': 0}


In [49]:
best_el = gcv.best_estimator_
print(elastic.coef_)
print(elastic.intercept_)

[ 8.77941715e-01 -1.05098333e-01  3.07404259e-01  1.06821865e-01
  1.42351777e-01 -1.96635254e-01 -1.40528099e+00  8.86557895e-02
 -1.28329740e+00 -7.00767490e-01 -8.31697745e-02  7.58873002e-01
  6.76422548e-02  4.14788044e-02 -8.05254411e-01  8.42850524e-02
 -3.09471846e-03  6.99847071e-02 -3.42279787e-02  4.40828730e-02
  3.74032430e-01  3.00742401e-01 -7.09392753e-01  7.92794746e-05
 -3.46429716e-01  1.62185880e-03  9.75858996e-03  1.19379915e-04
 -0.00000000e+00  4.33005602e-03 -8.22959857e-03 -4.42300551e-03
  3.74794239e-01  2.05171030e-02 -3.41485744e-02 -1.10335939e-02
 -4.79558090e-03  8.71785973e-04 -5.62856154e-03 -6.07994310e-02
  1.74497304e+00  1.65824226e-01  1.19811163e-01  2.71954913e-01
 -4.10234842e-01 -0.00000000e+00  3.73681969e-03  0.00000000e+00
 -6.02021242e-01 -5.22170053e-01  7.42330902e-02  0.00000000e+00
 -7.76718675e-02 -5.49561778e-02  2.45955502e-01  0.00000000e+00
  1.52204120e+00]
46.960184286215636


In [53]:
imp = SimpleImputer().set_output(transform='pandas')
pipe = Pipeline([('IMP',imp), ('ELASTIC', elastic)])
print(pipe.get_params())
params = {'ELASTIC__alpha':np.linspace(0.01, 10, 20),
         'ELASTIC__l1_ratio':np.linspace(0,1,10),
         'IMP__strategy':['mean','median']}
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold)
gcv.fit(X,y)
print(gcv.best_score_)
print(gcv.best_params_)

{'memory': None, 'steps': [('IMP', SimpleImputer()), ('ELASTIC', ElasticNet(alpha=0.001))], 'verbose': False, 'IMP': SimpleImputer(), 'ELASTIC': ElasticNet(alpha=0.001), 'IMP__add_indicator': False, 'IMP__copy': True, 'IMP__fill_value': None, 'IMP__keep_empty_features': False, 'IMP__missing_values': nan, 'IMP__strategy': 'mean', 'ELASTIC__alpha': 0.001, 'ELASTIC__copy_X': True, 'ELASTIC__fit_intercept': True, 'ELASTIC__l1_ratio': 0.5, 'ELASTIC__max_iter': 1000, 'ELASTIC__positive': False, 'ELASTIC__precompute': False, 'ELASTIC__random_state': None, 'ELASTIC__selection': 'cyclic', 'ELASTIC__tol': 0.0001, 'ELASTIC__warm_start': False}


C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.289e+01, tolerance: 4.974e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.716e+01, tolerance: 5.010e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers im

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.952e+01, tolerance: 4.543e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.560e+01, tolerance: 4.974e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.250e+01, tolerance: 4.554e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.174e+01, tolerance: 4.543e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.262e+01, tolerance: 4.974e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.472e+01, tolerance: 5.010e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers im

...

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.073e+00, tolerance: 5.010e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.279e+00, tolerance: 5.010e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.522e+01, tolerance: 4.554e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.201e+01, tolerance: 4.543e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.719e-01, tolerance: 4.974e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.504e+00, tolerance: 5.010e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

0.4669944159287545
{'ELASTIC__alpha': 0.5357894736842106, 'ELASTIC__l1_ratio': 0.1111111111111111, 'IMP__strategy': 'mean'}


C:\Users\dbda\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.505e+01, tolerance: 5.961e-02
  model = cd_fast.enet_coordinate_descent(


In [54]:
print(gcv.best_score_)
print(gcv.best_params_)

0.4669944159287545
{'ELASTIC__alpha': 0.5357894736842106, 'ELASTIC__l1_ratio': 0.1111111111111111, 'IMP__strategy': 'mean'}
